In [ ]:
!pip install plotly dash numpy

In [ ]:
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import numpy as np
import math

In [ ]:
def velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    T_K = T + 273.15
    T_ref_K = T_ref + 273.15
    T_opt_K = T_opt + 273.15

    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    exp2 = math.exp((deltaH * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    fator_f = math.exp(-((T_K - T_opt_K) / sigma)**2)

    numerador = Vmax_ref * exp1 * fator_f * S
    denominador = Km_ref * exp2 + S

    return term1 * (numerador / denominador)